In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

### First Data

In [2]:
# replace file path with the location of data_task on local computer memory
df = pd.read_excel(r"C:\Users\snair\Downloads\data_task.xlsx", sheet_name='order_numbers')

In [3]:
# calculating the difference between order numbers
df['order_number_difference'] = df['order_number'].diff()

# calculating the difference between the 2 subsequent dates which contain order numbers
df['date_difference'] = df['date'].diff()


In [4]:
df.shape

(856, 4)

In [5]:
df.head

<bound method NDFrame.head of           date  order_number  order_number_difference date_difference
0   2018-01-07      33841906                      NaN             NaT
1   2018-01-22      34008921                 167015.0         15 days
2   2018-01-25      34397468                 388547.0          3 days
3   2018-02-06      34434432                  36964.0         12 days
4   2018-02-08      34579365                 144933.0          2 days
..         ...           ...                      ...             ...
851 2022-12-26      89053562                  47918.0          1 days
852 2022-12-27      89078365                  24803.0          1 days
853 2022-12-28      89109007                  30642.0          1 days
854 2022-12-29      89139373                  30366.0          1 days
855 2022-12-30      89172576                  33203.0          1 days

[856 rows x 4 columns]>

In [6]:
# Converting 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Creating interactive plot with Plotly
fig = px.line(df, x='date', y='order_number', title='Order Number over Time', markers=True)

# Displaying plot
fig.show()

In [7]:
# Initialising the 'data_integrity' column with True (assuming all data is good initially)
df['data_integrity'] = True

# Iterating through each row and compare order_number with previous and next valid rows
for i in range(1, len(df) - 1):
    current_order = df.loc[i, 'order_number']
    previous_index = i - 1
    next_index = i + 1

    # Checking for previous valid row (where 'data_integrity' is True)
    while previous_index >= 0 and not df.loc[previous_index, 'data_integrity']:
        previous_index -= 1

    # Checking for next valid row (where 'data_integrity' is True)
    while next_index < len(df) and not df.loc[next_index, 'data_integrity']:
        next_index += 1

    # comparing the current row with previous and next good rows
    if previous_index >= 0 and next_index < len(df):
        previous_order = df.loc[previous_index, 'order_number']
        next_order = df.loc[next_index, 'order_number']

        # checking the condition
        if not (previous_order < current_order < next_order):
            df.loc[i, 'data_integrity'] = False 

In [8]:
df_good=df[df['data_integrity']==True]

In [9]:
df_good.shape

(498, 5)

In [10]:
# Converting 'date' column to datetime format
df_good['date'] = pd.to_datetime(df_good['date'], format='%d/%m/%Y')

# Creating interactive plot with Plotly
fig = px.line(df_good, x='date', y='order_number', title='Order Number over Time', markers=True)

# Displaying plot
fig.show()

In [11]:
# Taking max value in case of multiple values from a same date
df_good_2 = df_good.loc[df_good.groupby('date')['order_number'].idxmax()]
df_good_2.shape

(468, 5)

In [12]:
# Converting 'date' column to datetime format
df_good_2['date'] = pd.to_datetime(df_good_2['date'], format='%d/%m/%Y')

# Creating interactive plot with Plotly
fig = px.line(df_good, x='date', y='order_number', title='Order Number over Time', markers=True)

# Displaying the plot
fig.show()

In [13]:
df_good_2.shape

(468, 5)

### Second Data

In [14]:
# reading and loading the transaction data
df_spend = pd.read_excel(r"C:\Users\snair\Downloads\data_task.xlsx", sheet_name='transaction_data')

In [15]:
# finding the correlation between total_spend_index and weekly_active_users_index
col1 = 'total_spend_index'
col2 = 'weekly_active_users_index'

correlation = df_spend[col1].corr(df_spend[col2])

print(f"The correlation between {col1} and {col2} is: {correlation}")

The correlation between total_spend_index and weekly_active_users_index is: 0.7163052468998714


Because the feature weekly_active_users_index is highly correlated to total_spend_index, we won't be including this feature for training.

### Third Data

In [16]:
# loading the reported data on to a dataframe
df_period = pd.read_excel(r"C:\Users\snair\Downloads\data_task.xlsx", sheet_name='reported_data')

### Processing

In [17]:
# Converting 'date' columns to datetime format
df_good_2['date'] = pd.to_datetime(df_good_2['date'], format='%d/%m/%Y')
df_spend['date'] = pd.to_datetime(df_spend['date'], format='%d/%m/%Y')

# Initialising new columns in the third dataset for total orders and total spend index
df_period['total_orders'] = 0
df_period['total_spend_index'] = 0

# Iterating over each period in the third dataset
for index, row in df_period.iterrows():
    start_date = pd.to_datetime(row['start_date'], format='%d/%m/%Y')
    end_date = pd.to_datetime(row['end_date'], format='%d/%m/%Y')
    
    # Handling order numbers: Filter, interpolate, and calculate total orders
    complete_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    df_orders_filtered = df_good_2.set_index('date').reindex(complete_date_range)
    df_orders_filtered['order_number'] = df_orders_filtered['order_number'].interpolate(method='linear')
    df_orders_filtered['order_diff'] = df_orders_filtered['order_number'].diff()
    total_orders_for_period = df_orders_filtered['order_diff'].sum()

    # Handling spend index: Filter and calculate total spend index
    df_spend_filtered = df_spend[(df_spend['date'] >= start_date) & (df_spend['date'] <= end_date)]
    total_spend_for_period = df_spend_filtered['total_spend_index'].sum()

    # Adding calculated values to the third dataset
    df_period.at[index, 'total_orders'] = total_orders_for_period
    df_period.at[index, 'total_spend_index'] = total_spend_for_period

# Save the updated third dataset back to Excel
# df_period.to_excel('updated_third_data.xlsx', index=False)

In [18]:
data = df_period

Model 1

In [19]:
# Feature engineering
# Converting 'start_date' and 'end_date' into datetime and derive features
data['start_date'] = pd.to_datetime(data['start_date'], format='%d/%m/%Y')
data['end_date'] = pd.to_datetime(data['end_date'], format='%d/%m/%Y')

# Feature 1: Total days in the period
data['total_days'] = (data['end_date'] - data['start_date']).dt.days

# Feature 2: Extract the year of the start date
data['start_year'] = data['start_date'].dt.year

# Defining feature columns and target column
features = ['total_days', 'start_year', 'total_orders', 'total_spend_index']  
target = 'revenue_index'

# Splitting data into features (X) and target (y)
X = data[features]
y = data[target]

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialising the models
lasso_model = Lasso(alpha=0.1)
ridge_model = Ridge(alpha=0.1)
ridge_cv_model = RidgeCV(alphas=[0.1, 0.25, 0.5, 1.0, 10.0], cv=5)  # Ridge with Cross-Validation
gboost_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)


# Training the models
lasso_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
ridge_cv_model.fit(X_train, y_train)
gboost_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Generating predictions for the test set
lasso_preds = lasso_model.predict(X_test)
ridge_preds = ridge_model.predict(X_test)
ridge_cv_preds = ridge_cv_model.predict(X_test)
gboost_preds = gboost_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# Combining predictions using different weights
weights = {
    'lasso': 0.15,
    'ridge': 0.1,
    'ridge_cv': 0.15,
    'gboost': 0.1,
    'xgb':0.5
}

# Final prediction is a weighted average of individual model predictions
final_preds = (weights['lasso'] * lasso_preds +
               weights['ridge'] * ridge_preds +
               weights['ridge_cv'] * ridge_cv_preds +
               weights['gboost'] * gboost_preds +
               weights['xgb'] * xgb_preds )

# Evaluating the ensemble model using Mean Squared Error
mse = mean_squared_error(y_test, final_preds)
print(f"Ensemble Model Mean Squared Error: {mse}")

# Creating a Function to make predictions using the ensemble model
def predict_revenue_ensemble(start_date, end_date):
    # Converting start_date and end_date to datetime
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    
    # Calculating total days in the period
    total_days = (end_date - start_date).days
    
    # Extracting the year from the start date
    start_year = start_date.year
     
    # Creating a DataFrame for the input features (use placeholders for missing features)
    input_data = pd.DataFrame({
        'total_days': [total_days],
        'start_year': [start_year],
        'total_orders': [0],  # Placeholder or mean value
        'total_spend_index': [0]  # Placeholder or mean value
    })
    
    # Making predictions from individual models
    lasso_pred = lasso_model.predict(input_data)
    ridge_pred = ridge_model.predict(input_data)
    ridge_cv_pred = ridge_cv_model.predict(input_data)
    gboost_pred = gboost_model.predict(input_data)
    xgboost_pred = xgb_model.predict(input_data)
    
    # Combining predictions using the same weights
    final_pred = (weights['lasso'] * lasso_pred +
                  weights['ridge'] * ridge_pred +
                  weights['ridge_cv'] * ridge_cv_pred +
                  weights['gboost'] * gboost_pred + 
                  weights['xgb'] * xgboost_pred)
    
    return final_pred[0]


Ensemble Model Mean Squared Error: 6195.952803320154


In [20]:
#Predicting the revenue index
predicted_revenue = predict_revenue_ensemble('01/01/2022', '31/12/2022')
print(f"Predicted Revenue Index: {predicted_revenue}")

Predicted Revenue Index: 509.17351338549753


Model 2

In [25]:
# Feature engineering
# Converting 'start_date' and 'end_date' into datetime and derive features
data['start_date'] = pd.to_datetime(data['start_date'], format='%d/%m/%Y')
data['end_date'] = pd.to_datetime(data['end_date'], format='%d/%m/%Y')

# Feature 1: Total days in the period
data['total_days'] = (data['end_date'] - data['start_date']).dt.days

# Feature 2: Extract the year of the start date
data['start_year'] = data['start_date'].dt.year

# Defining feature columns and target column
features = ['total_days', 'start_year', 'total_spend_index']
target = 'revenue_index'

# Splitting data into features (X) and target (y)
X = data[features]
y = data[target]

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialising the models
lasso_model = Lasso(alpha=0.1)
ridge_model = Ridge(alpha=0.1)
ridge_cv_model = RidgeCV(alphas=[0.1, 0.25, 0.5, 1.0, 10.0], cv=5)  # Ridge with Cross-Validation
gboost_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)


# Training the models
lasso_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
ridge_cv_model.fit(X_train, y_train)
gboost_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Generating predictions for the test set
lasso_preds = lasso_model.predict(X_test)
ridge_preds = ridge_model.predict(X_test)
ridge_cv_preds = ridge_cv_model.predict(X_test)
gboost_preds = gboost_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# Combining predictions using different weights
weights = {
    'lasso': 0.15,
    'ridge': 0.1,
    'ridge_cv': 0.15,
    'gboost': 0.1,
    'xgb':0.5
}

# Final prediction is a weighted average of individual model predictions
final_preds = (weights['lasso'] * lasso_preds +
               weights['ridge'] * ridge_preds +
               weights['ridge_cv'] * ridge_cv_preds +
               weights['gboost'] * gboost_preds +
               weights['xgb'] * xgb_preds )

# Evaluating the ensemble model using Mean Squared Error
mse = mean_squared_error(y_test, final_preds)
print(f"Ensemble Model Mean Squared Error: {mse}")

# Creating a Function to make predictions using the ensemble model
def predict_revenue_ensemble_2(start_date, end_date):
    # Converting start_date and end_date to datetime
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    
    # Calculating total days in the period
    total_days = (end_date - start_date).days
    
    # Extracting the year from the start date
    start_year = start_date.year
     
    # Creating a DataFrame for the input features (use placeholders for missing features)
    input_data = pd.DataFrame({
        'total_days': [total_days],
        'start_year': [start_year],
        'total_spend_index': [0]  
    })
    
    # Making predictions from individual models
    lasso_pred = lasso_model.predict(input_data)
    ridge_pred = ridge_model.predict(input_data)
    ridge_cv_pred = ridge_cv_model.predict(input_data)
    gboost_pred = gboost_model.predict(input_data)
    xgboost_pred = xgb_model.predict(input_data)
    
    # Combining predictions using the same weights
    final_pred = (weights['lasso'] * lasso_pred +
                  weights['ridge'] * ridge_pred +
                  weights['ridge_cv'] * ridge_cv_pred +
                  weights['gboost'] * gboost_pred + 
                  weights['xgb'] * xgboost_pred)
    
    return final_pred[0]


Ensemble Model Mean Squared Error: 1947.1807916299167


In [26]:
predicted_revenue = predict_revenue_ensemble_2('01/01/2022', '31/12/2022')
print(f"Predicted Revenue Index: {predicted_revenue}")

Predicted Revenue Index: 507.02415643573954
